In [1]:
import numpy as np
import dill
from src.graph import Graph
from src.evolution_strategies import one_plus_lambda, tournament_selection
from src.population import Population
from src.arg_parser import parse_args

In [2]:
def bool_if(x, y, z): return x if z else y
def bool_and(x, y): return x and y
def bool_or(x, y): return x or y
def bool_not(x): return not x


Population.add_operation(arity=2, func=bool_and, string="AND")
Population.add_operation(arity=2, func=bool_or, string="OR")
Population.add_operation(arity=1, func=bool_not, string="NOT")
Population.add_operation(arity=3, func=bool_if, string="IF")

In [3]:
Population.rng = np.random.default_rng(10)

In [4]:
pop = Population(
    population_size = 1000,
    n_in = 11,
    n_out = 1,
    n_middle=256
)

In [16]:
def node_g_mark(g: Graph, n_index:int, depth: int, op_arr):
    if depth == 0:
        return op_arr[n_index]

    node = g.nodes[n_index]
    mark = ""
    for input in node.decode_inputs():
        mark += node_g_mark(g, input, depth-1, op_arr) + ", "
    mark = f"{op_arr[n_index]} ({mark[:-2]})"
    
    return mark

def graph_genetic_marks(g: Graph, depth = 1):
    op_arr = []
    for node in g.nodes:
        if node.operation == None:
            op_arr.append(f"n_{node.id}")
        else:
            op_arr.append(g.decode_operation(node.operation).string)


    markers: Dict[str, int] = {}
    for n_index, node in enumerate(g.nodes):
        if len(node.inputs) == 0 or node.operation == None:
            continue

        mark = node_g_mark(g, n_index, depth, op_arr)
        if mark not in markers:
            markers[mark] = [0, 0]

        if node.active:
            markers[mark][0] += 1
        else:
            markers[mark][1] += 1
    return markers

def population_genetic_marks(pop: Population, depth = 1):
    markers_by_indv = {}
    global_gen_markers = {}
    for indv in pop.indvs:
        g_mark = graph_genetic_marks(indv, depth)
        markers_by_indv[indv.id] = g_mark
        for mark, qtd in g_mark.items():
            if mark not in global_gen_markers:
                global_gen_markers[mark] = [0, 0]
            global_gen_markers[mark][0] += qtd[0]
            global_gen_markers[mark][1] += qtd[1]

    return markers_by_indv, global_gen_markers

In [20]:
_, global_gen_markers = population_genetic_marks(pop, 2)
active_cont = 0
inactive_cont = 0
for qtd in global_gen_markers.values():
    if qtd[0] > 0:
        active_cont += 1
    if qtd[1] > 0:
        inactive_cont += 1
global_gen_markers

{'OR (n_6 (), n_2 ())': [9, 19],
 'NOT (n_8 ())': [144, 703],
 'IF (n_3 (), n_5 (), n_5 ())': [1, 2],
 'AND (n_6 (), n_6 ())': [5, 18],
 'OR (n_4 (), OR (n_6, n_2))': [1, 0],
 'OR (n_8 (), n_1 ())': [4, 19],
 'IF (n_5 (), OR (n_6, n_2), n_7 ())': [1, 0],
 'OR (n_8 (), OR (n_4, OR))': [0, 1],
 'IF (n_5 (), OR (n_4, OR), NOT (n_8))': [1, 0],
 'NOT (IF (n_5, OR, n_7))': [2, 0],
 'OR (n_6 (), n_5 ())': [6, 16],
 'OR (IF (n_5, OR, NOT), n_0 ())': [1, 0],
 'IF (n_2 (), NOT (n_8), NOT (n_8))': [0, 1],
 'NOT (n_10 ())': [127, 694],
 'AND (OR (n_8, n_1), IF (n_2, NOT, NOT))': [0, 1],
 'NOT (n_4 ())': [124, 701],
 'IF (n_4 (), n_7 (), n_7 ())': [0, 1],
 'IF (OR (n_6, n_5), OR (n_6, n_2), IF (n_5, OR, NOT))': [0, 1],
 'NOT (OR (n_8, n_1))': [0, 10],
 'AND (n_10 (), n_2 ())': [6, 19],
 'NOT (n_1 ())': [106, 716],
 'OR (OR (n_8, n_1), n_9 ())': [0, 1],
 'IF (OR (n_6, n_5), n_8 (), OR (n_6, n_2))': [0, 1],
 'IF (AND (n_6, n_6), IF (n_2, NOT, NOT), NOT (n_1))': [0, 1],
 'AND (IF (n_5, OR, n_7), NOT (

In [7]:
# import cProfile
# import pstats
# def exec_func():
#     for i in range(100):
#         population_genetic_marks(pop)
# profile = cProfile.Profile()
# profile.runcall(exec_func)
# ps = pstats.Stats(profile)
# ps.print_stats()
# print()

         40300102 function calls in 22.890 seconds

   Random listing order was used

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  5100000    0.409    0.000    0.409    0.000 {method 'append' of 'list' objects}
    50000    0.012    0.000    0.012    0.000 {method 'items' of 'dict' objects}
 15100000    0.924    0.000    0.924    0.000 {built-in method builtins.len}
        1    0.202    0.202   22.890   22.890 /tmp/ipykernel_12667/3219028410.py:3(exec_func)
 10000000    3.116    0.000    3.719    0.000 /home/k1/faculdade/research/cgp/CGPython/src/graph.py:50(decode_operation)
  5000000    5.164    0.000    5.164    0.000 /home/k1/faculdade/research/cgp/CGPython/src/node.py:25(<listcomp>)
  5000000    1.804    0.000    6.968    0.000 /home/k1/faculdade/research/cgp/CGPython/src/node.py:24(decode_inputs)
    50000    9.870    0.000   21.286    0.000 /home/k1/faculdade/research/cgp/CGPython/src/diversity_measures.py:63(graph_genetic_marks)
      100    1.389 